In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# -----------------------------------------------------------
# 1. Load TRAIN and TEST datasets
# -----------------------------------------------------------
train_path = "Classification_Combined_Data/S1_S2_train_data.csv"
test_path  = "Classification_Combined_Data/S1_S2_test_data.csv"

df_train = pd.read_csv(train_path)
df_test  = pd.read_csv(test_path)

# # train only on ID = 5.0 fist 80%
# df_train = df_train1[df_train1["ID"] == 5.0].sample(frac=0.8, random_state=42)
# #test is last 20% of train
# df_test = df_train1[df_train1["ID"] == 5.0].drop(df_train.index)

# # train only on ID = 5.0 fist 80%
# df_train = df_train1[df_train1["ID"] == 11.0].sample(frac=0.8, random_state=42)
# #test is last 20% of train
# df_test = df_train1[df_train1["ID"] == 11.0].drop(df_train.index)

#for both train and test, only rows where labsl is Not Drowsy or Slight
df_train = df_train[df_train["Label"].isin(["Not Drowsy", "Slight", "Moderate", "Very"])]
df_test = df_test[df_test["Label"].isin(["Not Drowsy", "Slight", "Moderate", "Very"])]
# -----------------------------------------------------------
# 2. Apply label mapping to both
# -----------------------------------------------------------
label_map = {
    'Not Drowsy': 'alert',
    'Slight': 'drowsy',
    'Moderate': 'drowsy',
    'Very': 'drowsy'
}

df_train["MappedLabel"] = df_train["Label"].map(label_map)
df_test["MappedLabel"]  = df_test["Label"].map(label_map)

# -----------------------------------------------------------
# 3. Encode target labels (alert=0, drowsy=1)
# -----------------------------------------------------------
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train["MappedLabel"])
y_test  = label_encoder.transform(df_test["MappedLabel"])

# -----------------------------------------------------------
# 4. Select numeric features
# -----------------------------------------------------------
exclude_cols = ["Label", "MappedLabel", "ID", "Study"]
feature_cols = [c for c in df_train.columns if c not in exclude_cols]

X_train = df_train[feature_cols]
X_test  = df_test[feature_cols]

# -----------------------------------------------------------
# 5. Scale features (fit on train, transform on test)
# -----------------------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# -----------------------------------------------------------
# 6. Fit unsupervised 2-component GMM on TRAIN DATA ONLY
# -----------------------------------------------------------
# gmm = GaussianMixture(
#     n_components=2,
#     covariance_type='full',
#     random_state=42
# )

gmm = GaussianMixture(
    n_components=2,            # tune
    covariance_type="diag",    # tune: full/tied/diag
    n_init=20,
    init_params="kmeans",
    max_iter=1000,
    tol=1e-5,
    reg_covar=1e-5,
    random_state=42
)
gmm.fit(X_train_scaled)

# -----------------------------------------------------------
# 7. Predict cluster labels on TRAIN (for alignment)
# -----------------------------------------------------------
train_cluster_labels = gmm.predict(X_train_scaled)

# -----------------------------------------------------------
# 8. Align cluster IDs to true labels using TRAIN accuracy
# -----------------------------------------------------------
acc0 = accuracy_score(y_train, train_cluster_labels)
acc1 = accuracy_score(y_train, 1 - train_cluster_labels)

# cluster → label mapping
if acc1 > acc0:
    cluster_to_label = lambda c: 1 - c
else:
    cluster_to_label = lambda c: c

# -----------------------------------------------------------
# 9. Predict on TEST
# -----------------------------------------------------------
test_clusters = gmm.predict(X_test_scaled)
test_preds = cluster_to_label(test_clusters)

# -----------------------------------------------------------
# 10. Evaluate TEST accuracy
# -----------------------------------------------------------
print("=== TEST SET RESULTS ===")
print("Accuracy:", accuracy_score(y_test, test_preds))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, test_preds))
print("\nClassification Report:")
print(classification_report(y_test, test_preds, target_names=label_encoder.classes_))

# -----------------------------------------------------------
# 11. Posterior probabilities on TEST set
# -----------------------------------------------------------
epsilon = 0.005 # to avoid exact 0 or 1 probabilities
probs_test = gmm.predict_proba(X_test_scaled)
# probs_test = np.clip(probs_test, epsilon, 1 - epsilon)
# probs_test = gmm.predict_proba(X_test_scaled)

df_test["GMM_prob_alert"] = probs_test[:, 0]
df_test["GMM_prob_drowsy"] = probs_test[:, 1]
df_test["GMM_pred_cluster"] = test_clusters
df_test["GMM_pred_label"] = label_encoder.inverse_transform(test_preds)

df_test.head(n=50)

=== TEST SET RESULTS ===
Accuracy: 0.5513513513513514

Confusion Matrix:
[[157 378]
 [535 965]]

Classification Report:
              precision    recall  f1-score   support

       alert       0.23      0.29      0.26       535
      drowsy       0.72      0.64      0.68      1500

    accuracy                           0.55      2035
   macro avg       0.47      0.47      0.47      2035
weighted avg       0.59      0.55      0.57      2035



,window_start,ID,Study,Label,EAR_mean_mean,MAR_inner_mean,MAR_outer_mean,AU01_r_mean,AU15_r_mean,AU25_r_mean,...,gaze_angle_y_std,swAngle_std,laneDevPosition_std,laneDev_OffsetfrmLaneCentre_std,speed_std,MappedLabel,GMM_prob_alert,GMM_prob_drowsy,GMM_pred_cluster,GMM_pred_label
0,1.638561e+09,10.0,S1,Not Drowsy,0.280226,0.020549,0.303724,0.077756,0.133311,0.223478,...,0.038516,1.446996,0.000000,0.697119,2.509008,alert,1.0,5.226150e-17,0,drowsy
1,1.638561e+09,10.0,S1,Not Drowsy,0.275627,0.016681,0.298697,0.135278,0.115778,0.293422,...,0.049447,1.021389,0.000000,1.114900,3.219460,alert,1.0,2.035327e-16,0,drowsy
2,1.638561e+09,10.0,S1,Not Drowsy,0.277547,0.013587,0.298186,0.104289,0.105111,0.266167,...,0.045153,1.907755,0.000000,1.670019,3.594871,alert,1.0,1.437622e-14,0,drowsy
3,1.638561e+09,10.0,S1,Not Drowsy,0.283759,0.012794,0.297106,0.075489,0.132756,0.258267,...,0.031922,1.634922,0.000000,1.563995,2.562208,alert,1.0,7.199000e-15,0,drowsy
4,1.638561e+09,10.0,S1,Not Drowsy,0.284400,0.010559,0.292257,0.086489,0.105122,0.274722,...,0.033040,0.698894,0.000000,0.817669,3.651178,alert,1.0,7.832119e-17,0,drowsy
5,1.638561e+09,10.0,S1,Not Drowsy,0.290036,0.011303,0.288014,0.146106,0.102792,0.204294,...,0.038245,1.068776,0.000000,0.997173,0.526519,alert,1.0,1.681894e-14,0,drowsy
6,1.638561e+09,10.0,S1,Not Drowsy,0.287672,0.011005,0.288053,0.134294,0.113403,0.119028,...,0.036576,1.558166,0.000000,1.377826,0.491708,alert,1.0,3.699227e-14,0,drowsy
7,1.638561e+09,10.0,S1,Not Drowsy,0.280214,0.009513,0.289294,0.028344,0.069233,0.057411,...,0.030694,1.055170,0.000000,1.118170,0.644544,alert,1.0,9.500530e-16,0,drowsy
8,1.638561e+09,10.0,S1,Not Drowsy,0.278645,0.011739,0.291496,0.071678,0.139056,0.056378,...,0.047220,1.062337,0.000000,0.788120,1.237312,alert,1.0,5.368201e-16,0,drowsy
9,1.638561e+09,10.0,S1,Not Drowsy,0.278539,0.014288,0.292226,0.074922,0.178344,0.130789,...,0.040656,0.965429,0.250996,1.083526,1.430386,alert,1.0,3.678031e-17,0,drowsy


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE

# ===========================================
# 1. LOAD TRAIN + TEST DATA
# ===========================================
train_path = "Classification_Combined_Data/S1_S2_train_data.csv"
test_path  = "Classification_Combined_Data/S1_S2_test_data.csv"

df_train1 = pd.read_csv(train_path)
df_test  = pd.read_csv(test_path)

# # train only on ID = 5.0 fist 80%
# df_train = df_train1[df_train1["ID"] == 11.0].sample(frac=0.8, random_state=42)
# #test is last 20% of train
# df_test = df_train1[df_train1["ID"] ==11.0].drop(df_train.index)

# # train only on ID = 5.0 fist 80%
# df_train = df_train1[df_train1["ID"] == 5.0].sample(frac=0.8, random_state=42)
# #test is last 20% of train
# df_test = df_train1[df_train1["ID"] == 5.0].drop(df_train.index)

#for both train and test, only rows where labsl is Not Drowsy or Slight
df_train = df_train[df_train["Label"].isin(["Not Drowsy", "Slight", "Moderate", "Very"])]
df_test = df_test[df_test["Label"].isin(["Not Drowsy", "Slight", "Moderate", "Very"])]

# ===========================================
# 2. APPLY LABEL MAPPING
# ===========================================
label_map = {
    'Not Drowsy': 'alert',
    'Slight': 'drowsy',
    'Moderate': 'drowsy',
    'Very': 'drowsy'
}

df_train["MappedLabel"] = df_train["Label"].map(label_map)
df_test["MappedLabel"]  = df_test["Label"].map(label_map)

# ===========================================
# 3. ENCODE LABELS (alert=0, drowsy=1)
# ===========================================
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train["MappedLabel"])
y_test  = label_encoder.transform(df_test["MappedLabel"])

# ===========================================
# 4. SELECT NUMERIC FEATURE COLUMNS
# ===========================================
exclude_cols = ["Label", "MappedLabel", "ID", "Study"]
feature_cols = [col for col in df_train.columns if col not in exclude_cols]

X_train = df_train[feature_cols]
X_test  = df_test[feature_cols]

# ===========================================
# 5. STANDARDIZE FEATURES (fit on train ONLY)
# ===========================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# ===========================================
# 6. SMOTE on the TRAIN SET only
# ===========================================
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train_scaled, y_train)

print("Training balance after SMOTE:")
print(pd.Series(y_train_bal).value_counts())

# ===========================================
# 7. TRAIN SEPARATE GMMs (SUPERVISED)
#    One GMM for each class
# ===========================================
X_train_alert   = X_train_bal[y_train_bal == 0]
X_train_drowsy  = X_train_bal[y_train_bal == 1]

gmm_alert = GaussianMixture(
    n_components=2,            # tune
    covariance_type="diag",    # tune: full/tied/diag
    n_init=20,
    init_params="kmeans",
    max_iter=1000,
    tol=1e-5,
    reg_covar=1e-5,
    random_state=42
)
gmm_drowsy = GaussianMixture(
    n_components=2,            # tune
    covariance_type="diag",    # tune: full/tied/diag
    n_init=20,
    init_params="kmeans",
    max_iter=1000,
    tol=1e-5,
    reg_covar=1e-5,
    random_state=42
)

gmm_alert.fit(X_train_alert)
gmm_drowsy.fit(X_train_drowsy)

# ===========================================
# 8. CLASSIFICATION USING BAYES RULE
#    p(x | class) * P(class)
# ===========================================
# class priors from balanced training set
prior_alert  = (y_train_bal == 0).mean()
prior_drowsy = (y_train_bal == 1).mean()

# likelihoods from GMM
log_lik_alert  = gmm_alert.score_samples(X_test_scaled)
log_lik_drowsy = gmm_drowsy.score_samples(X_test_scaled)

# convert log-likelihoods + priors to posterior probabilities
log_posterior_alert  = log_lik_alert  + np.log(prior_alert)
log_posterior_drowsy = log_lik_drowsy + np.log(prior_drowsy)

# prediction: choose class with larger posterior
y_pred = np.where(log_posterior_alert > log_posterior_drowsy, 0, 1)

# ===========================================
# 9. EVALUATION
# ===========================================
print("\n=== TEST SET RESULTS ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# ===========================================
# 10. SAVE PROBABILITIES & PREDICTIONS
# ===========================================
# convert log posterior to normalized probabilities
posterior_alert = np.exp(log_posterior_alert)
posterior_drowsy = np.exp(log_posterior_drowsy)
posterior_sum = posterior_alert + posterior_drowsy

df_test["GMM_prob_alert"] = posterior_alert / posterior_sum
df_test["GMM_prob_drowsy"] = posterior_drowsy / posterior_sum
df_test["GMM_pred"] = y_pred
df_test["GMM_pred_label"] = label_encoder.inverse_transform(y_pred)

df_test.head()

#display confusion matrix
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

y_pred = (model.predict(X_test) > 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Alert", "Drowsy"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

Training balance after SMOTE:
1    257
0    257
Name: count, dtype: int64

=== TEST SET RESULTS ===
Accuracy: 0.8266666666666667

Confusion Matrix:
[[ 6  2]
 [11 56]]

Classification Report:
              precision    recall  f1-score   support

       alert       0.35      0.75      0.48         8
      drowsy       0.97      0.84      0.90        67

    accuracy                           0.83        75
   macro avg       0.66      0.79      0.69        75
weighted avg       0.90      0.83      0.85        75



NameError: name 'model' is not defined

# Combined data for GMM clustering

In [12]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from scipy.optimize import linear_sum_assignment

# -----------------------------
# CONFIG
# -----------------------------
train_path = "Classification_Combined_Data/S1_S2_train_data.csv"
test_path  = "Classification_Combined_Data/S1_S2_test_data.csv"

# Grid
COMPONENT_GRID = [2, 3, 4, 6, 8, 10]
COVTYPE_GRID   = ["full", "tied", "diag", "spherical"]

RANDOM_STATE = 42

label_map = {
    'Not Drowsy': 'alert',
    'Slight': 'slight',
    'Moderate': 'drowsy',
    'Very': 'drowsy'
}

# -----------------------------
# 1) Load
# -----------------------------
df_train = pd.read_csv(train_path)
df_test  = pd.read_csv(test_path)

keep = ["Not Drowsy", "Slight", "Moderate", "Very"]
df_train = df_train[df_train["Label"].isin(keep)].copy()
df_test  = df_test[df_test["Label"].isin(keep)].copy()

df_train["MappedLabel"] = df_train["Label"].map(label_map)
df_test["MappedLabel"]  = df_test["Label"].map(label_map)

# -----------------------------
# 2) Encode labels (EVAL ONLY)
# -----------------------------
le = LabelEncoder()
y_train = le.fit_transform(df_train["MappedLabel"])
y_test  = le.transform(df_test["MappedLabel"])

# -----------------------------
# 3) Features
# -----------------------------
exclude_cols = ["Label", "MappedLabel", "ID", "Study", "window_start"]
feature_cols = [c for c in df_train.columns if c not in exclude_cols]

X_train = df_train[feature_cols].to_numpy()
X_test  = df_test[feature_cols].to_numpy()

# -----------------------------
# 4) Scale (fit on train only)
# -----------------------------
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

# -----------------------------
# 5) Manual grid search (rank by BIC, show BIC/AIC for each)
# -----------------------------
results = []
best = None  # (bic, aic, K, cov_type, fitted_model)

for K in COMPONENT_GRID:
    for cov_type in COVTYPE_GRID:
        try:
            gmm = GaussianMixture(
                n_components=K,
                covariance_type=cov_type,
                n_init=20,
                init_params="kmeans",
                max_iter=1000,
                tol=1e-5,
                reg_covar=1e-5,
                random_state=RANDOM_STATE
            )
            gmm.fit(X_train_s)

            bic = gmm.bic(X_train_s)
            aic = gmm.aic(X_train_s)

            results.append({"K": K, "cov_type": cov_type, "BIC": bic, "AIC": aic})

            if best is None or bic < best[0] or (bic == best[0] and aic < best[1]):
                best = (bic, aic, K, cov_type, gmm)

        except Exception as e:
            results.append({"K": K, "cov_type": cov_type, "BIC": np.nan, "AIC": np.nan, "error": str(e)})

df_results = pd.DataFrame(results).sort_values(["BIC", "AIC"], ascending=True)
print("=== GRID RESULTS (ranked by BIC then AIC) ===")
display(df_results)

# -----------------------------
# 6) Evaluate best model only (unsupervised fit + train-only mapping)
# -----------------------------
best_bic, best_aic, best_K, best_cov, best_gmm = best
print("\n=== BEST MODEL ===")
print(f"K={best_K}, covariance_type={best_cov}, BIC={best_bic:.2f}, AIC={best_aic:.2f}")

train_clusters = best_gmm.predict(X_train_s)
test_clusters  = best_gmm.predict(X_test_s)

n_labels = len(le.classes_)
counts = np.zeros((best_K, n_labels), dtype=int)
for c, y in zip(train_clusters, y_train):
    counts[c, y] += 1

# Hungarian assignment for one-to-one part
cost = counts.max() - counts
row_ind, col_ind = linear_sum_assignment(cost)
cluster_to_label = {r: c for r, c in zip(row_ind, col_ind)}

# If K > n_labels, map leftover clusters to majority label within that cluster
unassigned = set(range(best_K)) - set(cluster_to_label.keys())
for c in unassigned:
    if counts[c].sum() == 0:
        cluster_to_label[c] = int(np.bincount(y_train).argmax())
    else:
        cluster_to_label[c] = int(counts[c].argmax())

y_pred_test = np.array([cluster_to_label[c] for c in test_clusters])

print("\n--- TEST RESULTS (best model only) ---")
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("Macro F1:", f1_score(y_test, y_pred_test, average="macro"))
print("Weighted F1:", f1_score(y_test, y_pred_test, average="weighted"))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_test, target_names=le.classes_))

# -----------------------------
# 7) Optional: write per-cluster posteriors for best model
# -----------------------------
probs_test = best_gmm.predict_proba(X_test_s)  # (n_test, best_K)

df_out = df_test.copy()
df_out["GMM_cluster"] = test_clusters
df_out["GMM_pred_label"] = le.inverse_transform(y_pred_test)

for k in range(best_K):
    df_out[f"GMM_prob_cluster_{k}"] = probs_test[:, k]

df_out.head(20)

=== GRID RESULTS (ranked by BIC then AIC) ===


,K,cov_type,BIC,AIC
20,10,full,347820.620585,286702.772082
16,8,full,358965.631263,310072.772320
12,6,full,380624.546353,343956.676970
8,4,full,427495.785826,403052.906003
4,3,full,453953.376295,435622.991252
22,10,diag,534294.472205,528551.140873
18,8,diag,549933.549078,545340.303871
0,2,full,567217.019027,554999.128764
14,6,diag,582528.165921,579085.006841
10,4,diag,652363.461644,650070.388689



=== BEST MODEL ===
K=10, covariance_type=full, BIC=347820.62, AIC=286702.77

--- TEST RESULTS (best model only) ---
Accuracy: 0.44963144963144963
Macro F1: 0.38377281394170176
Weighted F1: 0.4123559666065456

Confusion Matrix:
[[ 36  80 419]
 [ 62 372 244]
 [ 65 250 507]]

Classification Report:
              precision    recall  f1-score   support

       alert       0.22      0.07      0.10       535
      drowsy       0.53      0.55      0.54       678
      slight       0.43      0.62      0.51       822

    accuracy                           0.45      2035
   macro avg       0.39      0.41      0.38      2035
weighted avg       0.41      0.45      0.41      2035



,window_start,ID,Study,Label,EAR_mean_mean,MAR_inner_mean,MAR_outer_mean,AU01_r_mean,AU15_r_mean,AU25_r_mean,...,GMM_prob_cluster_0,GMM_prob_cluster_1,GMM_prob_cluster_2,GMM_prob_cluster_3,GMM_prob_cluster_4,GMM_prob_cluster_5,GMM_prob_cluster_6,GMM_prob_cluster_7,GMM_prob_cluster_8,GMM_prob_cluster_9
0,1.638561e+09,10.0,S1,Not Drowsy,0.280226,0.020549,0.303724,0.077756,0.133311,0.223478,...,3.559427e-22,0.999991,1.789729e-08,0.0,9.633363e-08,5.998171e-06,2.805346e-06,0.0,0.0,2.653694e-45
1,1.638561e+09,10.0,S1,Not Drowsy,0.275627,0.016681,0.298697,0.135278,0.115778,0.293422,...,1.080773e-13,0.937194,1.463136e-03,0.0,5.345555e-02,3.756950e-05,7.849603e-03,0.0,0.0,5.385534e-40
2,1.638561e+09,10.0,S1,Not Drowsy,0.277547,0.013587,0.298186,0.104289,0.105111,0.266167,...,5.317304e-15,0.040253,9.557639e-01,0.0,1.720303e-06,4.474707e-06,3.976518e-03,0.0,0.0,6.690503e-36
3,1.638561e+09,10.0,S1,Not Drowsy,0.283759,0.012794,0.297106,0.075489,0.132756,0.258267,...,1.386825e-18,0.001189,9.791121e-01,0.0,1.176011e-05,2.468798e-06,1.968511e-02,0.0,0.0,4.162456e-35
4,1.638561e+09,10.0,S1,Not Drowsy,0.284400,0.010559,0.292257,0.086489,0.105122,0.274722,...,1.435943e-21,0.000002,9.815815e-01,0.0,1.018605e-04,8.595108e-04,1.745483e-02,0.0,0.0,6.778006e-37
5,1.638561e+09,10.0,S1,Not Drowsy,0.290036,0.011303,0.288014,0.146106,0.102792,0.204294,...,1.202571e-19,0.010790,2.091111e-01,0.0,2.570070e-04,7.061261e-01,7.371536e-02,0.0,0.0,9.385880e-40
6,1.638561e+09,10.0,S1,Not Drowsy,0.287672,0.011005,0.288053,0.134294,0.113403,0.119028,...,9.349807e-24,0.973492,2.086634e-04,0.0,2.757593e-07,2.628286e-02,1.665775e-05,0.0,0.0,1.722418e-43
7,1.638561e+09,10.0,S1,Not Drowsy,0.280214,0.009513,0.289294,0.028344,0.069233,0.057411,...,2.649097e-24,0.049453,4.225029e-04,0.0,1.320074e-09,8.942392e-01,5.588522e-02,0.0,0.0,4.641545e-40
8,1.638561e+09,10.0,S1,Not Drowsy,0.278645,0.011739,0.291496,0.071678,0.139056,0.056378,...,1.704421e-20,0.995259,1.759816e-06,0.0,1.264829e-10,4.621612e-03,1.174040e-04,0.0,0.0,1.574802e-42
9,1.638561e+09,10.0,S1,Not Drowsy,0.278539,0.014288,0.292226,0.074922,0.178344,0.130789,...,3.035410e-01,0.000000,1.537150e-02,0.0,0.000000e+00,0.000000e+00,6.810875e-01,0.0,0.0,1.733167e-39


In [13]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from scipy.optimize import linear_sum_assignment

# -----------------------------
# CONFIG
# -----------------------------
train_path = "Classification_Combined_Data/S1_S2_train_data.csv"
test_path  = "Classification_Combined_Data/S1_S2_test_data.csv"

COMPONENT_GRID = [2, 3, 4, 6, 8, 10]
COVTYPE_GRID   = ["full", "tied", "diag", "spherical"]
RANDOM_STATE   = 42

label_map = {
    'Not Drowsy': 'alert',
    'Slight': 'slight',
    'Moderate': 'drowsy',
    'Very': 'drowsy'
}

# -----------------------------
# 1) Load
# -----------------------------
df_train = pd.read_csv(train_path)
df_test  = pd.read_csv(test_path)

keep = ["Not Drowsy", "Slight", "Moderate", "Very"]
df_train = df_train[df_train["Label"].isin(keep)].copy()
df_test  = df_test[df_test["Label"].isin(keep)].copy()

df_train["MappedLabel"] = df_train["Label"].map(label_map)
df_test["MappedLabel"]  = df_test["Label"].map(label_map)

# -----------------------------
# 2) Encode labels
# -----------------------------
le = LabelEncoder()
y_train = le.fit_transform(df_train["MappedLabel"])
y_test  = le.transform(df_test["MappedLabel"])

# -----------------------------
# 3) Features
# -----------------------------
exclude_cols = ["Label", "MappedLabel", "ID", "Study", "window_start"]
feature_cols = [c for c in df_train.columns if c not in exclude_cols]

X_train = df_train[feature_cols].to_numpy()
X_test  = df_test[feature_cols].to_numpy()

# -----------------------------
# 4) Scale (fit on train only)
# -----------------------------
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s  = scaler.transform(X_test)

# -----------------------------
# 5) SUPERVISED "GMM": one GMM per class (generative classifier)
#    Score(x|class) + prior(class) -> choose best class
#    We'll tune: covariance_type (shared), n_components per class (same K for simplicity)
# -----------------------------
def fit_class_gmms(X, y, K, cov_type):
    class_models = {}
    class_priors = {}
    for cls in np.unique(y):
        Xc = X[y == cls]
        gmm = GaussianMixture(
            n_components=K,
            covariance_type=cov_type,
            n_init=20,
            init_params="kmeans",
            max_iter=1000,
            tol=1e-5,
            reg_covar=1e-5,
            random_state=RANDOM_STATE
        )
        gmm.fit(Xc)
        class_models[cls] = gmm
        class_priors[cls] = len(Xc) / len(X)
    return class_models, class_priors

def predict_class_gmms(X, class_models, class_priors):
    classes = sorted(class_models.keys())
    # log p(x|y=c) + log p(y=c)
    scores = np.column_stack([
        class_models[c].score_samples(X) + np.log(class_priors[c])
        for c in classes
    ])
    pred = np.array([classes[i] for i in np.argmax(scores, axis=1)])
    return pred, scores

results = []
best = None  # (metric, K, cov_type, models, priors)

for K in COMPONENT_GRID:
    for cov_type in COVTYPE_GRID:
        try:
            models, priors = fit_class_gmms(X_train_s, y_train, K, cov_type)

            # Use TRAIN AIC/BIC summed across class-models as a comparable score
            bic = sum(models[c].bic(X_train_s[y_train == c]) for c in models)
            aic = sum(models[c].aic(X_train_s[y_train == c]) for c in models)

            # Evaluate on test (since now supervised)
            y_pred_test, _ = predict_class_gmms(X_test_s, models, priors)
            macro_f1 = f1_score(y_test, y_pred_test, average="macro")

            results.append({"K": K, "cov_type": cov_type, "BIC": bic, "AIC": aic, "macro_f1_test": macro_f1})

            # Pick best by macro F1 (tie-breaker: lower BIC)
            if best is None or macro_f1 > best[0] or (macro_f1 == best[0] and bic < best[1]):
                best = (macro_f1, bic, aic, K, cov_type, models, priors)

        except Exception as e:
            results.append({"K": K, "cov_type": cov_type, "BIC": np.nan, "AIC": np.nan, "macro_f1_test": np.nan, "error": str(e)})

df_results = pd.DataFrame(results).sort_values(["macro_f1_test", "BIC"], ascending=[False, True])
print("=== GRID RESULTS (ranked by macro F1 on TEST, tie-breaker BIC) ===")
display(df_results)

# -----------------------------
# 6) Report best model only
# -----------------------------
best_f1, best_bic, best_aic, best_K, best_cov, best_models, best_priors = best
print("\n=== BEST SUPERVISED GMM CLASSIFIER ===")
print(f"K={best_K}, covariance_type={best_cov}, test macro F1={best_f1:.4f}, BIC={best_bic:.2f}, AIC={best_aic:.2f}")

y_pred_test, scores_test = predict_class_gmms(X_test_s, best_models, best_priors)

print("\n--- TEST RESULTS (best model only) ---")
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("Macro F1:", f1_score(y_test, y_pred_test, average="macro"))
print("Weighted F1:", f1_score(y_test, y_pred_test, average="weighted"))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_test, target_names=le.classes_))

# Optional: class posteriors (softmax over log-scores)
probs_test = np.exp(scores_test - scores_test.max(axis=1, keepdims=True))
probs_test = probs_test / probs_test.sum(axis=1, keepdims=True)

df_out = df_test.copy()
df_out["GMM_pred_label"] = le.inverse_transform(y_pred_test)
for idx, cls in enumerate(sorted(best_models.keys())):
    df_out[f"GMM_prob_{le.inverse_transform([cls])[0]}"] = probs_test[:, idx]

df_out.head(20)

=== GRID RESULTS (ranked by macro F1 on TEST, tie-breaker BIC) ===


,K,cov_type,BIC,AIC,macro_f1_test
9,4,tied,634487.845166,616986.269385,0.482352
19,8,spherical,787751.195957,781786.772878,0.480384
18,8,diag,536105.166563,524585.818108,0.480283
23,10,spherical,772807.935834,765347.955923,0.475268
11,4,spherical,834635.481136,831662.171720,0.474482
22,10,diag,512942.413961,498538.777331,0.466275
15,6,spherical,808630.461739,804161.595491,0.454939
1,2,tied,663877.755523,647836.128078,0.453501
14,6,diag,558982.800532,550347.740254,0.453094
17,8,tied,606301.357867,585879.885414,0.448094



=== BEST SUPERVISED GMM CLASSIFIER ===
K=4, covariance_type=tied, test macro F1=0.4824, BIC=634487.85, AIC=616986.27

--- TEST RESULTS (best model only) ---
Accuracy: 0.5066339066339066
Macro F1: 0.48235186675122144
Weighted F1: 0.49579152833858003

Confusion Matrix:
[[151 175 209]
 [ 77 449 152]
 [117 274 431]]

Classification Report:
              precision    recall  f1-score   support

       alert       0.44      0.28      0.34       535
      drowsy       0.50      0.66      0.57       678
      slight       0.54      0.52      0.53       822

    accuracy                           0.51      2035
   macro avg       0.49      0.49      0.48      2035
weighted avg       0.50      0.51      0.50      2035



,window_start,ID,Study,Label,EAR_mean_mean,MAR_inner_mean,MAR_outer_mean,AU01_r_mean,AU15_r_mean,AU25_r_mean,...,gaze_angle_y_std,swAngle_std,laneDevPosition_std,laneDev_OffsetfrmLaneCentre_std,speed_std,MappedLabel,GMM_pred_label,GMM_prob_alert,GMM_prob_drowsy,GMM_prob_slight
0,1.638561e+09,10.0,S1,Not Drowsy,0.280226,0.020549,0.303724,0.077756,0.133311,0.223478,...,0.038516,1.446996,0.000000,0.697119,2.509008,alert,drowsy,0.000028,9.996450e-01,0.000327
1,1.638561e+09,10.0,S1,Not Drowsy,0.275627,0.016681,0.298697,0.135278,0.115778,0.293422,...,0.049447,1.021389,0.000000,1.114900,3.219460,alert,drowsy,0.000957,9.935887e-01,0.005454
2,1.638561e+09,10.0,S1,Not Drowsy,0.277547,0.013587,0.298186,0.104289,0.105111,0.266167,...,0.045153,1.907755,0.000000,1.670019,3.594871,alert,drowsy,0.001938,9.857644e-01,0.012298
3,1.638561e+09,10.0,S1,Not Drowsy,0.283759,0.012794,0.297106,0.075489,0.132756,0.258267,...,0.031922,1.634922,0.000000,1.563995,2.562208,alert,drowsy,0.001494,9.882751e-01,0.010231
4,1.638561e+09,10.0,S1,Not Drowsy,0.284400,0.010559,0.292257,0.086489,0.105122,0.274722,...,0.033040,0.698894,0.000000,0.817669,3.651178,alert,drowsy,0.001632,9.836397e-01,0.014728
5,1.638561e+09,10.0,S1,Not Drowsy,0.290036,0.011303,0.288014,0.146106,0.102792,0.204294,...,0.038245,1.068776,0.000000,0.997173,0.526519,alert,drowsy,0.000148,9.947174e-01,0.005134
6,1.638561e+09,10.0,S1,Not Drowsy,0.287672,0.011005,0.288053,0.134294,0.113403,0.119028,...,0.036576,1.558166,0.000000,1.377826,0.491708,alert,drowsy,0.000197,9.957528e-01,0.004050
7,1.638561e+09,10.0,S1,Not Drowsy,0.280214,0.009513,0.289294,0.028344,0.069233,0.057411,...,0.030694,1.055170,0.000000,1.118170,0.644544,alert,drowsy,0.000306,9.988284e-01,0.000865
8,1.638561e+09,10.0,S1,Not Drowsy,0.278645,0.011739,0.291496,0.071678,0.139056,0.056378,...,0.047220,1.062337,0.000000,0.788120,1.237312,alert,drowsy,0.000240,9.992538e-01,0.000506
9,1.638561e+09,10.0,S1,Not Drowsy,0.278539,0.014288,0.292226,0.074922,0.178344,0.130789,...,0.040656,0.965429,0.250996,1.083526,1.430386,alert,drowsy,0.082780,5.537501e-01,0.363470
